# Store Sales Analysis
### Time Series Forecasting of Store Sales

The project aims to forecast sales for a large grocery retailer which sells thousands of products at Favorita stores in Ecuador's west coast. Favorita Corporation has a diverse portfolio, operating in various sectors across Ecuador and six other countries in the region.  There are 54 stores, 33 product types, and the data ranges over 56 months.

In [1]:
# import libraries 

import pandas as pd
import numpy as np
import calendar
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.offline as offline
import plotly.graph_objs as go
from scipy.interpolate import interp1d
import matplotlib.colors as mcolors
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error


offline.init_notebook_mode(connected=True)

Below we create a summary of the data.  This allows us to understand the data we are working with and how to best analyze it.

In [2]:
import plotly.graph_objs as go

def create_store_sales_summary():
    # Initialize a new figure
    fig = go.Figure()

    # Add big numbers text
    fig.add_trace(create_big_numbers_trace())

    # Add description labels
    fig.add_trace(create_description_trace())

    # Add horizontal lines
    add_horizontal_lines(fig)

    # Apply global layout settings
    apply_global_layout(fig)
    
    return fig

def create_big_numbers_trace():
    return go.Scatter(
        x=[0, 1, 2, 3],
        y=[1.6] * 4,
        mode="text",
        text=["<span style='font-size:33px'><b>54</b></span>",
              "<span style='font-size:33px'><b>33</b></span>",
              "<span style='font-size:33px'><b>16</b></span>",
              "<span style='font-size:33px'><b>56</b></span>"],
        textposition="bottom center"
    )

def create_description_trace():
    return go.Scatter(
        x=[0, 1, 2, 3],
        y=[1.1] * 4,
        mode="text",
        text=["Stores", "Products", "States", "Months"],
        textposition="bottom center"
    )

def add_horizontal_lines(fig):
    fig.add_hline(y=2.2, line_width=5, line_color='black')
    fig.add_hline(y=0.3, line_width=3, line_color='black')

def apply_global_layout(fig):
    fig.update_yaxes(visible=False)
    fig.update_xaxes(visible=False)
    fig.update_layout(
        showlegend=False,
        height=300,
        width=700,
        title='Store Sales Summary',
        title_x=0.5,
        title_y=0.9,
        xaxis_range=[-0.5, 3.6],
        yaxis_range=[-0.2, 2.2],
        plot_bgcolor='#fafafa',
        paper_bgcolor='#fafafa',
        font=dict(size=23, color='black'),
        title_font=dict(size=35, color='black'),
        margin=dict(t=90, l=70, b=0, r=70)
    )

# To create and display the figure
fig = create_store_sales_summary()
fig.show()


In [3]:
#import data

df_holi = pd.read_csv('./holidays_events.csv')
df_oil = pd.read_csv('./oil.csv')
df_stores = pd.read_csv('./stores.csv')
df_trans = pd.read_csv('./transactions.csv')

df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')

Below we combine our various datasets into a dataframe to allow for further data analysis.

In [4]:
def merge_and_process_dataframes(train_df, holi_df, oil_df, stores_df, trans_df):
    """
    Merge multiple dataframes and preprocess the resulting dataframe.

    Args:
    train_df: DataFrame containing the training data
    holi_df: DataFrame containing holiday information
    oil_df: DataFrame containing oil prices
    stores_df: DataFrame containing store information
    trans_df: DataFrame containing transaction data

    Returns:
    df_train1: Merged and preprocessed DataFrame
    """
  
    # Merge the training data with holiday, oil, stores, and transactions data
    df_train1 = (train_df
                .merge(holi_df, on='date', how='left')
                .merge(oil_df, on='date', how='left')
                .merge(stores_df, on='store_nbr', how='left')
                .merge(trans_df, on=['date', 'store_nbr'], how='left'))

    # Rename columns to avoid ambiguity
    df_train1.rename(columns={"type_x": "holiday_type", "type_y": "store_type"}, inplace=True)

    # Convert date column to datetime format
    df_train1['date'] = pd.to_datetime(df_train1['date'])
  
    # Create new columns for different time metrics
    df_train1['year'] = df_train1['date'].dt.year
    df_train1['month'] = df_train1['date'].dt.month
    df_train1['week'] = df_train1['date'].dt.isocalendar().week
    df_train1['quarter'] = df_train1['date'].dt.quarter
    df_train1['day_of_week'] = df_train1['date'].dt.day_name()

    return df_train1

# Assuming all other DataFrames (df_holi, df_oil, df_stores, df_trans) are defined
df_train1 = merge_and_process_dataframes(df_train, df_holi, df_oil, df_stores, df_trans)


### Overview of Average Sales
The following code creates an overview of average sales for different data types.  This will give us insight into what requires further analysis, and a quick overview of key statistics.

In [5]:
def create_sales_analysis_figure(df_train1):
    # Prepare data
    df_st_sa, df_fa_sa, df_cl_sa = prepare_data(df_train1)
    
    # Initialize subplot figure
    fig = initialize_subplots()
    
    # Add bar, pie, and bar charts
    add_family_sales_bar(fig, df_fa_sa)
    add_store_sales_pie(fig, df_st_sa)
    add_cluster_sales_bar(fig, df_cl_sa)
    
    # Update the global layout and styling
    update_global_style(fig, df_cl_sa)
    
    fig.show()
    

def prepare_data(df_train1):
    df_st_sa = df_train1.groupby('store_type').agg({"sales": "mean"}).reset_index().sort_values(by='sales', ascending=False)
    df_fa_sa = df_train1.groupby('family').agg({"sales": "mean"}).reset_index().sort_values(by='sales', ascending=False)[:10]
    df_cl_sa = df_train1.groupby('cluster').agg({"sales": "mean"}).reset_index()
    
    df_fa_sa['color'] = '#c6ccd8'
    df_fa_sa['color'][:2] = '#496595'
    df_cl_sa['color'] = '#c6ccd8'
    
    return df_st_sa, df_fa_sa, df_cl_sa


In [6]:
def initialize_subplots():
    fig = make_subplots(rows=2, cols=2,
                        specs=[[{"type": "bar"}, {"type": "pie"}],
                               [{"colspan": 2}, None]],
                        column_widths=[0.7, 0.3],
                        vertical_spacing=0,
                        horizontal_spacing=0.02)
    
    # Custom annotation to handle title text
    title_annotations=[
        dict(x=0, y=1.08, xref='paper', yref='paper', text='Top 10 Products with the Highest Sales', showarrow=False, font=dict(size=14)),
        dict(x=1, y=1.08, xref='paper', yref='paper', text='Stores with the Highest Sales', showarrow=False, font=dict(size=14)),
        dict(x=0.5, y=0.5, xref='paper', yref='paper', text='Clusters Vs Sales', showarrow=False, font=dict(size=14))
    ]
    
    fig.update_layout(annotations=title_annotations)
    return fig


def add_family_sales_bar(fig, df_fa_sa):
    fig.add_trace(go.Bar(x=df_fa_sa['sales'], y=df_fa_sa['family'],
                         marker=dict(color='#FADADD'),
                         name='Family', orientation='h'), 
                  row=1, col=1)
    
def add_store_sales_pie(fig, df_st_sa):
    fig.add_trace(go.Pie(values=df_st_sa['sales'], labels=df_st_sa['store_type'],
                         marker=dict(colors=['#FFC1E0', '#FFE4E1', '#FFCCCB', '#F4C2C2', '#FADADD']),
                         hole=0.7, hoverinfo='label+percent+value', textinfo='label'),
                  row=1, col=2)
    
def add_cluster_sales_bar(fig, df_cl_sa):
    fig.add_trace(go.Bar(x=df_cl_sa['cluster'], y=df_cl_sa['sales'],
                         marker=dict(color='#F8BBD0'),
                         name='Cluster'),
                  row=2, col=1)

def update_global_style(fig, df_cl_sa):
    fig.update_yaxes(showgrid=False, ticksuffix=' ', categoryorder='total ascending', row=1, col=1)
    fig.update_xaxes(visible=False, row=1, col=1)
    fig.update_xaxes(tickmode='array', tickvals=df_cl_sa['cluster'], ticktext=[i for i in range(1, 17)], row=2, col=1)
    fig.update_yaxes(visible=False, row=2, col=1)
    fig.update_layout(height=500, bargap=0.2,
                      margin=dict(b=0, r=20, l=20),
                      title_text="Analysis of Average Sales",
                      template="plotly_white",
                      title_font=dict(size=24, color='black'),
                      font=dict(color='black'),
                      hoverlabel=dict(bgcolor="#f2f2f2", font_size=13, font_family="Lato, sans-serif"),
                      showlegend=False)
# create figure
create_sales_analysis_figure(df_train1)

/var/folders/6z/144p_sbd6xgf3v73hnbcprww0000gn/T/ipykernel_57564/976925096.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Interpretation of Average Sales

The chart gives us a clear visual of what products sell the most, what stores have the highest sales, and what clusters have the most sales.

Grocery and Beverages have by far the most sales.
Store Type A leads in sales.
Cluster 5 has the most sales.

## Average Sales over Time

The following section analyzes the average sales over time using weekly, monthly, and yearly dividers to visualize what times of year have the most sales, how sales have progressed through the years, and what days of week are the best for sales.

In [7]:
def create_yearly_sales_data(df_train1):
    # Create a dictionary to hold data for each year
    yearly_data = {}
    max_length = 0
    for year in range(2013, 2018):
        df_year = df_train1[df_train1['year'] == year][['month', 'sales']]
        year_sales = df_year.groupby('month').agg({"sales": "mean"}).reset_index()['sales'].tolist()
        
        # Track the maximum length
        max_length = max(max_length, len(year_sales))
        yearly_data[f's{year}'] = year_sales
    
    # Padding each list to the max_length
    for year, sales in yearly_data.items():
        yearly_data[year] = sales + [0] * (max_length - len(sales))
        
    return yearly_data

def add_bars(fig, x_data, y_data, colors):
    # Add bar plots
    for i in range(len(x_data[0])):
        for xd, yd in zip(x_data, y_data):
            fig.add_trace(go.Bar(
                x=[xd[i]], y=[yd],
                orientation='h',
                marker=dict(
                    color=colors[i],
                    line=dict(color='rgb(0, 0, 0)', width=1)  # Black line
                )
            ))

def add_annotations(fig, x_data, y_data, top_labels):
    # Add annotations
    annotations = []
    for yd, xd in zip(y_data, x_data):
        annotations.append(dict(xref='paper', yref='y',
                                x=0.14, y=yd,
                                xanchor='right',
                                text=str(yd),
                                font=dict(size=14, color='rgb(0, 0, 0)'),  # Black text
                                showarrow=False, align='right'))
        if yd == y_data[-1]:
            for i, label in enumerate(top_labels):
                annotations.append(dict(xref='x', yref='paper',
                                        x=sum(xd[:i+1]) - xd[i]/2, y=1.1,
                                        text=label,
                                        font=dict(size=14, color='rgb(0, 0, 0)'),  # Black text
                                        showarrow=False))
    fig.update_layout(annotations=annotations)


In [8]:
# Data preparation
yearly_data = create_yearly_sales_data(df_train1)
df_year = pd.DataFrame(yearly_data)
x_data = df_year.values

# Color and label setup
colors = ['#D1C4E9', '#F3E5F5', '#F8BBD0', '#E1BEE7', '#C5CAE9']  # Pastel colors
top_labels = ['2013', '2014', '2015', '2016', '2017']
y_data = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Create figure
fig = go.Figure()
add_bars(fig, x_data, y_data, colors)
add_annotations(fig, x_data, y_data, top_labels)

# Update layout
fig.update_layout(title='Average Sales by Year',
                  xaxis=dict(showgrid=False, zeroline=False, domain=[0.15, 1]),
                  yaxis=dict(showgrid=False, showline=False, showticklabels=False, zeroline=False),
                  barmode='stack', template="plotly_white", margin=dict(l=0, r=50, t=100, b=10),
                  showlegend=False)
fig.show()


### Interpretation of Average Sales by Year

Note: there is no data for September to December of 2017.

December is consistently the best month for sales.  Other months have varied year to year, with February being a consistently low sale month and October and November being consistently higher than average sale months.

In [9]:
# Data Preparation for Monthly, Weekly, and Quarterly Sales
def prepare_data(df):
    df_m_sa = df.groupby('month').agg({"sales": "mean"}).reset_index()
    df_m_sa['sales'] = round(df_m_sa['sales'], 2)
    df_m_sa['month_text'] = df_m_sa['month'].apply(lambda x: calendar.month_abbr[x])
    
    df_w_sa = df.groupby('week').agg({"sales": "mean"}).reset_index()
    df_q_sa = df.groupby('quarter').agg({"sales": "mean"}).reset_index()
    
    return df_m_sa, df_w_sa, df_q_sa

# Prepare Data
df_m_sa, df_w_sa, df_q_sa = prepare_data(df_train1)

# Define Colors 
bar_colors = ['#D1C4E9', '#F3E5F5', '#F8BBD0', '#E1BEE7', '#C5CAE9', '#D4A5D4', '#F9CFB2', '#B3E0F2', '#FFABAB', '#ACE5FF', '#FFC3A0', '#CDB3D4']
pie_colors = ['#D1C4E9', '#F3E5F5', '#F8BBD0', '#E1BEE7', '#C5CAE9']
line_color = '#D1C4E9'

# Create Subplots
fig = make_subplots(rows=2, cols=2, specs=[[{"type": "bar"}, {"type": "pie"}], [{"colspan": 2}, None]],
                    subplot_titles=("Analysis of Average Sales per Month", "Analysis of Average Sales per Quarter", 
                                    "Analysis of Average Sales per Week"))

# Add Monthly Sales Bar Chart
fig.add_trace(go.Bar(x=df_m_sa['sales'], y=df_m_sa['month_text'], orientation='h',
                     marker=dict(color=bar_colors), name='Month'), row=1, col=1)

# Add Quarterly Sales Pie Chart
fig.add_trace(go.Pie(values=df_q_sa['sales'], labels=df_q_sa['quarter'],
                     marker=dict(colors=pie_colors), hole=0.7, textinfo='label+percent'), row=1, col=2)

# Add Weekly Sales Line Chart
fig.add_trace(go.Scatter(x=df_w_sa['week'], y=df_w_sa['sales'], mode='lines+markers',
                         fill='tozeroy', fillcolor=line_color, marker=dict(color='#4B0082'), name='Week'), row=2, col=1)

# Style Customization
fig.update_layout(height=750, bargap=0.15, title_text="Average Sales Analysis",
                  title_font=dict(size=25, color='black'), font=dict(color='black'), 
                 plot_bgcolor='#F5F5F5', paper_bgcolor='#F5F5F5', showlegend=False)

# Show Figure
fig.show()


### Interpretation of Average Sales over Time

By Month: December is by far the best month for sales.  November and July are nearly tied for 2nd/ 3rd highest sale months.  February is by far the worst month for sales.  

By Quarter:  Q4 is the best quarter for sales.  Q1, Q2, and Q3 are relatively similar in terms of average sales.

By Week:  Sales have fluctuated throughout the weeks with a slight upward trend.  

In [10]:
# Prepare data by grouping by 'day_of_week' and calculating the mean sales
df_dw_sa = df_train1.groupby('day_of_week').agg({"sales": "mean"}).reset_index()
df_dw_sa['sales'] = round(df_dw_sa['sales'], 2)

# Define the color palette 
colors = ['#D1C4E9', '#F3E5F5', '#F8BBD0', '#E1BEE7', '#C5CAE9', '#D4A5D4', '#F9CFB2']

# Custom sorting to ensure data appears in the right order
custom_order = {'Monday': 6, 'Tuesday': 5, 'Wednesday': 4, 'Thursday': 3, 'Friday': 2, 'Saturday': 1, 'Sunday': 0}
df_dw_sa['day_order'] = df_dw_sa['day_of_week'].map(custom_order)
df_dw_sa = df_dw_sa.sort_values('day_order')

# Create the bar chart
fig = go.Figure(data=[
    go.Bar(
        y=df_dw_sa['day_of_week'],
        x=df_dw_sa['sales'],
        text=df_dw_sa['sales'],
        marker_color=colors,
        orientation='h')
])


# Update layout 
fig.update_layout(
    title='Average Sales by Day of Week',
    template='plotly_white',
    height=340,
    yaxis=dict(showgrid=False, title=""),
    xaxis=dict(visible=False),
    title_font=dict(size=20, color='#000000'), 
    font=dict(color='#000000')  
)

# Show plot
fig.show()


### Interpretation of Average Sales by Day of Week

Highest sales are made on the weekend, particularly on Sunday. Thursday is the lowest day of the week for sales.

In [11]:
# Data preparation
# Grouping the data by 'store_type' and 'holiday_type' and taking the mean of 'sales'
df_st_ht = df_train1.groupby(['store_type', 'holiday_type']).agg({"sales": "mean"}).reset_index()
df_st_ht['sales'] = round(df_st_ht['sales'], 2)

# Create a list of colors for the scatter plot
graph_colors= ['#FFC0CB', '#DCA8C5', '#B990BE', '#9689B8', '#7471B1', '#525AAA', '#00008B']



# Generate the scatter plot
fig = px.scatter(df_st_ht, 
                 x='store_type', 
                 y='holiday_type', 
                 size='sales', 
                 color='sales',
                 color_continuous_scale=graph_colors,  
                 title="Average Sales for Store Types During Holidays")

# Styling
fig.update_yaxes(ticksuffix='  ')  
fig.update_layout(
    height=400, 
    xaxis_title='', 
    yaxis_title='',
    margin=dict(b=0),
    plot_bgcolor='#fafafa',  
    paper_bgcolor='#fafafa',  
    title_font=dict(size=29, color='#444', family="Lato, sans-serif"), 
    font=dict(color='#444'), 
    hoverlabel=dict(bgcolor="#f2f2f2", font_size=13, font_family="Lato, sans-serif")  # Hover label styling
)
fig.update_coloraxes(colorbar_title="Sales")

# Show the plot
fig.show()


### Interpretation of Average Sales for Store Types During Holidays

Store Type A consistently is a leader in sales across all Holiday Types, and Store Type C is consistently the lowest sales.  The Holiday Types with the best sales regardless of store type are Tranfer, and Additional .  The Holiday Types with the worst sales regardless of store type are Holiday, and Work Day. 

In [12]:
# Prepare data: Aggregate sales data by year, month, and store type
df_y_m_st = df_train1.groupby(['year', 'month', 'store_type']).agg({"sales": "mean"}).reset_index()
df_y_m_st['sales'] = round(df_y_m_st['sales'], 2)

# Define colors and labels
custom_colors = ['#FFC0CB', '#DCA8C5', '#B990BE', '#9689B8', '#7471B1', '#525AAA', '#00008B']
months_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Generate scatter plot
fig = px.scatter(df_y_m_st, 
                 x='month', 
                 y='store_type', 
                 color='sales', 
                 size='sales', 
                 color_continuous_scale=custom_colors,
                 facet_row='year', 
                 title='Average Sales for Store Types Each Month and Year',
                 labels={'store_type': 'Store Type', 'sales': 'Sales'})  # Update axis labels

# Update facet titles to show only the year
for annotation in fig['layout']['annotations']: 
    annotation['text'] = annotation['text'].split("=")[-1]

# Styling
fig.update_yaxes(ticksuffix='  ')
fig.update_xaxes(tickmode='array', tickvals=list(range(1, 13)), ticktext=months_labels)
fig.update_layout(height=900,
                  xaxis_title='', 
                  yaxis_title='', 
                  margin=dict(t=70, b=0),
                  plot_bgcolor='#fafafa', 
                  paper_bgcolor='#fafafa',
                  title_font=dict(size=22, color='#444', family="Lato, sans-serif"),
                  font=dict(color='#444'),
                  hoverlabel=dict(bgcolor="#f2f2f2", font_size=13, font_family="Lato, sans-serif"))

# Show the plot
fig.show()

### Interpretation of Average Sales for Store Types by Month and Year

This confirms our earlier findings that:
1. Store Type A is consistently a leader in sales
2. Sales have gradually and consistently increased each year for all Store Types & Months
3. December is consistently the best month for sales

In [13]:
# Data preparation
# Group by 'month' and 'holiday_type', and calculate the mean of 'sales'
df_m_ht = df_train1.groupby(['month', 'holiday_type']).agg({"sales": "mean"}).reset_index()
df_m_ht['sales'] = round(df_m_ht['sales'], 2)

# Color palette
color_palette = ['#FFC0CB', '#DCA8C5', '#B990BE', '#9689B8', '#7471B1', '#525AAA', '#00008B']

# Chart
# Use Plotly Express to generate a scatter plot
fig = px.scatter(df_m_ht,
                 x='month',
                 y='holiday_type',
                 size='sales',
                 color='sales',
                 color_continuous_scale=color_palette,  # Custom colors
                 title="Average Sales by Holiday Type Each Month",
                 labels={'sales': 'Sales'})  # Update label for sales

# Styling
# Customize axis ticks, layout, and other visual elements
fig.update_yaxes(ticksuffix='  ')
fig.update_xaxes(tickmode='array',
                 tickvals=list(range(1, 13)),
                 ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
fig.update_layout(height=400,
                  xaxis_title='',
                  yaxis_title='',
                  margin=dict(b=0),
                  plot_bgcolor='#fafafa',
                  paper_bgcolor='#fafafa',
                  title_font=dict(size=22, color='#444', family="Lato, sans-serif"),
                  font=dict(color='#444'),
                  hoverlabel=dict(bgcolor="#f2f2f2", font_size=13, font_family="Lato, sans-serif"))

# Show the chart
fig.show()

### Interpretation of Average Sales by Holiday Type Each Month

The majority of sales occurred during Transfer Holiday, with December and January being peak months likely due to the Christmas holidays. Additionally, November and May also demonstrated strong shopping trends.

In [14]:
# Data Preparation
# Group by 'year', 'month', and 'holiday_type' to find the mean of 'sales'
df_y_m_ht = df_train1.groupby(['year', 'month', 'holiday_type']).agg({'sales': 'mean'}).reset_index()
df_y_m_ht['sales'] = round(df_y_m_ht['sales'], 2)

# Generate Scatter Plot
fig = px.scatter(
    df_y_m_ht, 
    x='month', 
    y='holiday_type', 
    color='sales', 
    size='sales',
    facet_row='year', 
    title='Average Sales by Holiday Type and Year',
    color_continuous_scale=color_palette,
    labels={'holiday_type': 'Holiday Type', 'sales': 'Sales'}
)

# Styling
fig.update_yaxes(ticksuffix='  ')
fig.update_xaxes(
    tickmode='array', 
    tickvals=list(range(1, 13)),
    ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
)
fig.update_layout(
    height=1050,
    xaxis_title='',
    yaxis_title='',
    margin=dict(t=70, b=0),
    plot_bgcolor='#fafafa',
    paper_bgcolor='#fafafa',
    title_font=dict(size=22, color='#444', family="Lato, sans-serif"),
    font=dict(color='#444'),
    hoverlabel=dict(bgcolor="#f2f2f2", font_size=13, font_family="Lato, sans-serif")
)

for annotation in fig['layout']['annotations']:
    annotation['text'] = annotation['text'].replace('year=', '')

# Display Plot
fig.show()


## Data Preparation, Model Building, Forecasting and Evaluation

The following code creates a model that aims to be a useful tool for forecasting sales.

We start by preparing the data and then build an ARIMA model to forecast future sales.  An ARIMA (AutoRegressive Integrated Moving Average) model was chosen for its strengths in analyzing and forecasting time-series data- it is capable of capturing a suite of different time-based patterns, such as trends and seasonality, as well as accounting for error, making it a versatile and commonly used method for forecasting metrics like sales.   

In [15]:
# Make sure the data is sorted by time and reset the index
df_train1_sorted = df_train1.sort_values('date')
df_train1_sorted.reset_index(drop=True, inplace=True)

# Use the 'sales' column for ARIMA
sales_data = df_train1_sorted['sales']

# Build the ARIMA model
model = ARIMA(sales_data, order=(5,1,0))
model_fit = model.fit()

# Forecast future sales
future_steps = 12  # Number of steps to forecast
forecast = model_fit.forecast(steps=future_steps)


In [16]:
sales_data = df_train1['sales']

# Split into training and test sets
train_size = int(len(sales_data) * 0.8)
train, test = sales_data[0:train_size], sales_data[train_size:]

# Fit ARIMA model on training data
model = ARIMA(train, order=(5,1,0))
model_fit = model.fit()

# Forecast future sales for the size of test set
predictions = model_fit.forecast(steps=len(test))

# Evaluate the forecast with test set
# Adding a small constant to avoid division by zero in MAPE
rmse = np.sqrt(mean_squared_error(test, predictions))
mae = mean_absolute_error(test, predictions)


# Print evaluation metrics
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")


Root Mean Squared Error (RMSE): 1419.4861692091756
Mean Absolute Error (MAE): 480.5513712587709


### Model Performance Interpretation

The ARIMA model produced a Root Mean Squared Error (RMSE) of approximately 1419.49 and a Mean Absolute Error (MAE) of around 480.55.

RMSE: This value tells us that, on average, the model's forecasts are about 1419.49 units away from the actual sales figures when both are squared and then square-rooted. A lower RMSE is usually better, but the value needs to be evaluated relative to the range of the sales data.

MAE: This metric says that, on average, the model's predictions are about 480.55 units away from the actual sales values in absolute terms.

Both these metrics provide an idea of the model's accuracy. In the context of sales forecasting, lower values for RMSE and MAE would suggest a more accurate model. 

To fully interpret the quality of this model, these numbers need to be compared against the scale and variance of the data, which we do below.

In [17]:
# Assuming df_train1['sales'] contains your sales data
sales_data = df_train1['sales']

# Calculate the minimum and maximum values
min_value = sales_data.min()
max_value = sales_data.max()

# Calculate the range
data_range = max_value - min_value

print(f"Minimum Sales Value: {min_value}")
print(f"Maximum Sales Value: {max_value}")
print(f"Range of Sales Data: {data_range}")


Minimum Sales Value: 0.0
Maximum Sales Value: 124717.0
Range of Sales Data: 124717.0


### Model Performance Interpretation

Given the range of our sales data, which is from 0 to 124,717 (a range of 124,717 units), the RMSE of 1419.49 and MAE of 480.55 are relatively small in comparison. This suggests that the ARIMA model is quite accurate in the context of the overall range of our data.

Here's a quick breakdown:

RMSE of 1419.49: Compared to the maximum sales value of 124,717, the RMSE value is about 1.1% of the maximum sales value. This shows that the model's average squared error is fairly small relative to the range of sales.

MAE of 480.55: This value is even smaller, about 0.4% of the maximum sales value, showing that the model is doing quite well in terms of average absolute errors.

In summary, both RMSE and MAE suggest that the ARIMA model has fairly good accuracy in predicting future sales, considering the range of the data we have. 